In [9]:
import ray
from ray.rllib.algorithms.dqn import DQNConfig
from ray.tune.registry import register_env
from ray.rllib.env.env_context import EnvContext
import pandas as pd
import numpy as np
from gym_anytrading.envs import StocksEnv
from sklearn.preprocessing import MinMaxScaler
import os

# Initialize Ray
ray.init(ignore_reinit_error=True)

## 1. Data import and data scaling

df_train = pd.read_csv('data/df_train.csv', index_col=0)
df_train.index = pd.to_datetime(df_train.index)

df_test = pd.read_csv('data/df_test.csv', index_col=0)
df_test.index = pd.to_datetime(df_test.index)

# Removing duplicates in index (if any)
df_train = df_train[~df_train.index.duplicated(keep='first')]
df_test = df_test[~df_test.index.duplicated(keep='first')]

# Scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))

df_train = pd.DataFrame(scaler.fit_transform(df_train), columns=df_train.columns, index=df_train.index)
df_test = pd.DataFrame(scaler.transform(df_test), columns=df_test.columns, index=df_test.index)

# Features to use
features = ['rsi', 'ema', 'roc']

2024-09-19 10:42:25,815	INFO worker.py:1786 -- Started a local Ray instance.


In [12]:
## 2. Create the custom environment

def add_signals(env):
	start = env.frame_bound[0] - env.window_size
	end = env.frame_bound[1]
	prices = env.df.loc[:, 'Price'].to_numpy()[start:end]
	signal_features = env.df.loc[:, features].to_numpy()[start:end]

	return prices, signal_features


class MyCustomEnv(StocksEnv):
	_process_data = add_signals


def my_env_creator(env_config: EnvContext):
	return MyCustomEnv(df=df_train, window_size=12, frame_bound=(12, 440473))

register_env("custom_stock_env", my_env_creator)

In [20]:
config = (
    DQNConfig()
    .environment("custom_stock_env", env_config={})
    .framework("torch")
    .rollouts(num_rollout_workers=1)
    .training(lr=1e-3, gamma=0.99)
    .resources(num_gpus=0)
    .logging(log_dir="./ray_results")  # Custom log directory
)

2024-09-19 10:52:51,832	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!


AttributeError: 'DQNConfig' object has no attribute 'logging'

In [17]:
result = trainer.train()

In [ ]:
## 4. Evaluation

env_test = MyCustomEnv(df=df_test, window_size=12, frame_bound=(12, 224933))
obs = env_test.reset()

# Perform evaluation
done = False
while not done:
	action = trainer.compute_single_action(obs)
	obs, reward, done, info = env_test.step(action)

## 5. Store the results, Create plots

from helper_code.metrics_calculation import *

model_name = 'test'

final_results = calculate_metrics_create_plots(env_train, env_test, df_test, [], short_ticks=[], long_ticks=[],
											   model_name=model_name)
print(final_results)

# Shutdown Ray
ray.shutdown()